In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import platform

from matplotlib import font_manager, rc

plt.style.available
#plt.style.use('gadfly')

font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family = font_name)
    
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import numpy as np
import pandas as pd
import os

import folium
import json

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#os.chdir('C:/Users/yewoo/Anaconda3/Project/머신러닝_ver2.0')
os.chdir('D:\소음예측프로젝트\머신러닝2_화이팅...!!\data')

import mapboxgl
from mapboxgl.viz import *
import geopandas as gpd
from mapboxgl.utils import df_to_geojson
from mapboxgl.utils import(create_numeric_stops, create_color_stops, create_radius_stops)

# mapbox용 토큰 및 지도 center
token = 'pk.eyJ1IjoiYmptMDM2MSIsImEiOiJjazc0ZjJvcGswNGxrM2twbGtibTlxM3RrIn0._y5hcO3xGUtAEsocRw7tqA'
center = [126.986, 37.565]

## 1. 파일 읽기

In [2]:
data = pd.read_csv('200316_확장 통합 소음측정 데이터_ver.02_인코딩완료.csv',
                   encoding='utf-8', index_col=0)
data

,용도구분,법적구분,도로폭,도로명,도로규모,구,lng_x,lat_x,LEQ,2번신호등,...,비만율,스트레스 인지율,우울감 경험률,양호한 주관적 건강수준 인지율,가로녹시율평균,간선도로,보조간선도로,집산도로,국지도로,요일별통행속도평균
0,1,2,6,92,4,20,126.981299,37.573645,54.666667,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
1,1,2,6,92,4,20,126.981299,37.573645,52.166667,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
2,1,2,6,92,4,20,126.981299,37.573645,54.166667,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
3,1,2,6,92,4,20,126.981299,37.573645,52.833333,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
4,1,2,0,96,1,20,126.983676,37.573718,54.666667,5,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389207,7,0,0,17,1,0,127.054344,37.490652,74.420000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333
389208,7,0,0,17,1,0,127.054344,37.490652,74.670000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333
389209,7,0,0,17,1,0,127.054344,37.490652,74.410000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333
389210,7,0,0,17,1,0,127.054344,37.490652,73.920000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333


In [3]:
data.shape

(389212, 45)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 389212 entries, 0 to 389211
Data columns (total 45 columns):
용도구분                  389212 non-null int64
법적구분                  389212 non-null int64
도로폭                   389212 non-null int64
도로명                   389212 non-null int64
도로규모                  389212 non-null int64
구                     389212 non-null int64
lng_x                 389212 non-null float64
lat_x                 389212 non-null float64
LEQ                   389212 non-null float64
2번신호등                 389212 non-null int64
3번신호등                 389212 non-null int64
7번신호등                 389212 non-null int64
23번신호등                389212 non-null int64
27번신호등                389212 non-null int64
37번신호등                389212 non-null int64
237번신호등               389212 non-null int64
모든신호등                 389212 non-null int64
camera                389212 non-null int64
교통사고_발생건수             389212 non-null float64
교통사고_자동차 1만대당 발생건수    389212 non-null float64

#### 인코딩 기준

용도구분  
Index(['일반주거지역', '상업지역', '학교지역', '준공업지역', '종합병원지역', '전용주거지역', '준주거지역', '녹지지역'], dtype='object')
Int64Index([2, 1, 7, 5, 4, 3, 6, 0], dtype='int64')
  
법적구분  
Index(['나', '가', '다'], dtype='object')
Int64Index([1, 0, 2], dtype='int64')  

도로폭  
Index(['6m미만', '폭6-8m', '폭12-15m', '폭8-10m', '폭10-12m', '폭20-25m', '폭15-20m',
       '폭30-35m'],
      dtype='object')
Int64Index([0, 6, 2, 7, 1, 4, 3, 5], dtype='int64')  

도로명  
Index(['신반포로', '올림픽로', '목동서로', '종암로', '논현로', '신촌로', '경인로', '선릉로', '동일로215길',
       '대학로',
       ...
       '도봉로39길', '봉은사로18길', '강서로45다길', '국사봉2바길', '화곡로13길', '올림픽로70길',
       '구천면로24길', '잠원로', '북아현로', '퇴계로26길'],
      dtype='object', length=120)
Int64Index([ 69,  88,  36, 102,  20,  72,   6,  52,  70,  27,
            ...
             25,  13,   2, 116,  42,  89,  12,  99,  45, 112],
           dtype='int64', length=120)  
            
도로규모  
Index(['소로', '소로3류', '중로3류', '소로2류', '소로1류', '중로1류', '중로2류', '대로2류'], dtype='object')
Int64Index([1, 4, 7, 3, 2, 5, 6, 0], dtype='int64')  

구  
Index(['강남구', '성북구', '서초구', '구로구', '종로구', '영등포구', '강북구', '동대문구', '용산구', '은평구',
       '중구', '노원구', '중랑구', '관악구', '마포구', '송파구', '동작구', '양천구', '강서구', '강동구',
       '서대문구', '성동구', '광진구'],
      dtype='object')
Int64Index([ 0, 14, 12, 20, 17,  6,  2,  8, 18, 19, 21,  4,  7, 22, 10, 15,  9,
            16,  3,  1, 11, 13,  5],
           dtype='int64')

## 2. 머신러닝

### ========= 전체 변수 ==========

###  1. 종속변수, 독립변수 나누기

In [5]:
X = data.drop('LEQ', axis=1)
y = data.LEQ

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3,
                                                    random_state=0)

In [7]:
print('X_train :', X_train.shape)
print('X_test :', X_test.shape)
print('y_train :', y_train.shape)
print('y_test :', y_test.shape)

X_train : (272448, 44)
X_test : (116764, 44)
y_train : (272448,)
y_test : (116764,)


### 2. Scale 해주기

In [8]:
from sklearn.preprocessing import StandardScaler
st_scaler = StandardScaler().fit(X_train)

# 적용
X_train_scaled = st_scaler.transform(X_train)
X_test_scaled = st_scaler.transform(X_test)

In [9]:
print('X_train_scaled :', X_train_scaled.shape)
print('X_test_scaled :', X_test_scaled.shape)
print('y_train :', y_train.shape)
print('y_test :', y_test.shape)

X_train_scaled : (272448, 44)
X_test_scaled : (116764, 44)
y_train : (272448,)
y_test : (116764,)


### 3. 모델 빌딩 (XG Boost) 

In [10]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [11]:
xg_reg = xgb.XGBRegressor()

In [12]:
xg_reg.fit(X_train_scaled,y_train)

preds = xg_reg.predict(X_test_scaled)
preds

array([54.16211 , 62.25335 , 63.61414 , ..., 48.80581 , 57.612976,
       58.14867 ], dtype=float32)

In [13]:
print('예측 최대치: ', preds.max())
print('예측 최저치: ', preds.min())

예측 최대치:  76.41382
예측 최저치:  45.00347


### 4. 성능 확인 (RMSE, Accuracy)

In [14]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 2.167105


In [15]:
print('[ XG Boost Regression ]')
print("훈련 세트의 정확도 : {:.2f}".format(xg_reg.score(X_train_scaled, y_train)*100), '%')
print("테스트 세트의 정확도 : {:.2f}".format(xg_reg.score(X_test_scaled, y_test)*100), '%')

[ XG Boost Regression ]
훈련 세트의 정확도 : 93.58 %
테스트 세트의 정확도 : 93.56 %


### 5. 파라메터 조정으로 성능을 높여보자 !

In [16]:
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import datetime
from sklearn.model_selection import GridSearchCV

In [17]:
xgb1 = XGBRegressor()
parameters = {'nthread':[4,5], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [800]}

In [18]:
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

In [19]:
xgb_grid.fit(X_train_scaled, y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  36 out of  36 | elapsed: 42.4min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constrain...
                                    validate_parameters=False, verbosity=None),
             iid='warn', n_jobs=5,
             param_grid={'colsample_bytree': [0.7],
                         'learning_rate': [0.03, 0.05, 0.07],
                         'max_depth': [5, 6, 7], 'min_child_weight': [4],
                         '

In [20]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

0.9356738268884038
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 800, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [21]:
model_upgrade = XGBRegressor(colsample_bytree= 0.7, 
                             learning_rate= 0.07, 
                             max_depth= 7, 
                             min_child_weight= 4, 
                             n_estimators= 500, 
                             nthread= 4, 
                             objective= 'reg:linear', 
                             silent= 1, 
                             subsample= 0.7)

In [24]:
model_upgrade.fit(X_train_scaled, y_train)

# accuracy
print("훈련 세트의 정확도 : {:.3f}".format(model_upgrade.score(X_train_scaled, y_train)*100), '%')
print("테스트 세트의 정확도 : {:.3f}".format(model_upgrade.score(X_test_scaled, y_test)*100), '%')

훈련 세트의 정확도 : 93.577 %
테스트 세트의 정확도 : 93.556 %


### 필요한 공부

1. score()함수가 어떤 스코어인지 확인할 필요가 있다.
2. mape(mean absolute percentage error)로 성능을 평가해 볼 필요가 있다.
3. 그 외 다른 평가 방법을 찾아볼 필요가 있다.

### ========= 20개 변수 ==========

###  1. 종속변수, 독립변수 나누기

In [ ]:
X_20 = data.drop(['LEQ',
                      '3번신호등', '도로명', '2번신호등', '237번신호등', '23번신호등',
                       '교통사고_인구 10만명당 사망자수', '국지도로', '구', '집산도로', '간선도로', 
                       '소음배출시설_면적 합', '교통사고_인구 10만명당 부상자수',
                       '교통사고_사망자수', '우울감 경험률', '연간 고위험음주율', 
                       '교통사고_자동차 1만대당 발생건수', '면적', '인구밀도(명/㎢)', 
                       '현재흡연율', '비만율', '스트레스 인지율', '요일별통행속도평균',
                       '양호한 주관적 건강수준 인지율', '월간음주율'], axis=1)
y = data.LEQ

In [ ]:
X_20.columns

Index(['용도구분', '법적구분', '도로폭', '도로규모', 'lng_x', 'lat_x', '7번신호등', '27번신호등',
       '37번신호등', '모든신호등', 'camera', '교통사고_발생건수', '교통사고_부상자수', '인구',
       '소음배출시설_개소 합', '금연시도율', '증등도 이상 신체활동 실천율', '걷기실천율', '가로녹시율평균',
       '보조간선도로'],
      dtype='object')

In [ ]:
data

,용도구분,법적구분,도로폭,도로명,도로규모,구,lng_x,lat_x,LEQ,2번신호등,...,비만율,스트레스 인지율,우울감 경험률,양호한 주관적 건강수준 인지율,가로녹시율평균,간선도로,보조간선도로,집산도로,국지도로,요일별통행속도평균
0,1,2,6,92,4,20,126.981299,37.573645,54.666667,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
1,1,2,6,92,4,20,126.981299,37.573645,52.166667,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
2,1,2,6,92,4,20,126.981299,37.573645,54.166667,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
3,1,2,6,92,4,20,126.981299,37.573645,52.833333,0,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
4,1,2,0,96,1,20,126.983676,37.573718,54.666667,5,...,26.085714,28.885714,7.557143,44.242857,19.471667,21.636667,29.311667,16.395000,10.548333,23.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389207,7,0,0,17,1,0,127.054344,37.490652,74.420000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333
389208,7,0,0,17,1,0,127.054344,37.490652,74.670000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333
389209,7,0,0,17,1,0,127.054344,37.490652,74.410000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333
389210,7,0,0,17,1,0,127.054344,37.490652,73.920000,4,...,21.285714,30.400000,6.814286,50.714286,37.266667,41.136667,37.448333,39.736667,30.750000,24.683333


### 2. train, test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_20, y, test_size=.3,
                                                    random_state=0)

In [ ]:
print('X_train :', X_train.shape)
print('X_test :', X_test.shape)
print('y_train :', y_train.shape)
print('y_test :', y_test.shape)

X_train : (272448, 20)
X_test : (116764, 20)
y_train : (272448,)
y_test : (116764,)


### 2. Scale 해주기

In [3]:
from sklearn.preprocessing import StandardScaler
st_scaler = StandardScaler().fit(X_train)

# 적용
X_train_scaled = st_scaler.transform(X_train)
X_test_scaled = st_scaler.transform(X_test)

NameError: ignored

In [ ]:
print('X_train_scaled :', X_train_scaled.shape)
print('X_test_scaled :', X_test_scaled.shape)
print('y_train :', y_train.shape)
print('y_test :', y_test.shape)

### 3. 모델 빌딩 (XG Boost) 

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
xg_reg = xgb.XGBRegressor()

In [ ]:
xg_reg.fit(X_train_scaled,y_train)

preds = xg_reg.predict(X_test_scaled)
preds

array([54.15883 , 62.25331 , 63.612305, ..., 48.80572 , 57.612633,
       58.149544], dtype=float32)

### 4. 성능 확인 (RMSE, Accuracy)

In [1]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

NameError: ignored

In [2]:
print('[ XG Boost Regression ]')
print("훈련 세트의 정확도 : {:.2f}".format(xg_reg.score(X_train_scaled, y_train)*100), '%')
print("테스트 세트의 정확도 : {:.2f}".format(xg_reg.score(X_test_scaled, y_test)*100), '%')

[ XG Boost Regression ]


NameError: ignored

### 5. 파라메터 조정으로 성능을 높여보자 !

In [ ]:
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import datetime
from sklearn.model_selection import GridSearchCV

model_upgrade = XGBRegressor(colsample_bytree= 0.7, 
                             learning_rate= 0.07, 
                             max_depth= 7, 
                             min_child_weight= 4, 
                             n_estimators= 800, 
                             nthread= 4, 
                             objective= 'reg:linear', 
                             silent= 1, 
                             subsample= 0.7)

model_upgrade.fit(X_train_scaled, y_train)

# accuracy
print("훈련 세트의 정확도 : {:.3f}".format(model_upgrade.score(X_train_scaled, y_train)*100), '%')
print("테스트 세트의 정확도 : {:.3f}".format(model_upgrade.score(X_test_scaled, y_test)*100), '%')

In [ ]:
model_upgrade = XGBRegressor(colsample_bytree= 0.7, 
                             learning_rate= 0.07, 
                             max_depth= 7, 
                             min_child_weight= 4, 
                             n_estimators= 800, 
                             nthread= 4, 
                             objective= 'reg:linear', 
                             silent= 1, 
                             subsample= 0.7)

In [ ]:
model_upgrade.fit(X_train_scaled, y_train)

# accuracy
print("훈련 세트의 정확도 : {:.3f}".format(model_upgrade.score(X_train_scaled, y_train)*100), '%')
print("테스트 세트의 정확도 : {:.3f}".format(model_upgrade.score(X_test_scaled, y_test)*100), '%')

훈련 세트의 정확도 : 93.577 %
테스트 세트의 정확도 : 93.555 %


#### 모델 저장

In [ ]:
from sklearn.externals import joblib 

# 객체를 pickled binary file 형태로 저장한다 
file_name = 'model_upgrade.pkl' 
joblib.dump(model_upgrade, file_name) 

C:\Users\yewoo\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['model_upgrade.pkl']